In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x57f8e000 @  0x7f97b194d2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
!pip install -U -q PyDrive

import tensorflow as tf
import timeit

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

0.4.1
True


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Imports here
import torch
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import models

import cv2
import helper

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [0]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

# convert data to a normalized torch.FloatTensor
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

valid_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder('drive/My Drive/AI/flower_data/train', transform=train_transforms)
valid_data = datasets.ImageFolder('drive/My Drive/AI/flower_data/valid', transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainLoader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True) 
validLoader = torch.utils.data.DataLoader(valid_data, batch_size=32)

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
images,labels = next(iter(trainLoader))
print(images)

tensor([[[[-0.7479, -0.8849, -0.9705,  ...,  0.6392,  0.5878,  0.7591],
          [-0.7308, -0.8849, -0.9705,  ...,  0.7077,  0.6221,  0.5536],
          [-0.7308, -0.9020, -1.0048,  ...,  0.7077,  0.6906,  0.6221],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -1.6384, -1.6213, -1.6042],
          [-2.1179, -2.1179, -2.1179,  ..., -1.6384, -1.6213, -1.6042],
          [-2.1179, -2.1179, -2.1179,  ..., -1.6384, -1.6213, -1.6213]],

         [[-1.0553, -1.2129, -1.3004,  ..., -0.4601, -0.4951, -0.2850],
          [-1.0378, -1.2129, -1.3004,  ..., -0.4951, -0.5476, -0.5651],
          [-1.0378, -1.2304, -1.3354,  ..., -0.4951, -0.5126, -0.5476],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -1.5630, -1.5455, -1.5280],
          [-2.0357, -2.0357, -2.0357,  ..., -1.5630, -1.5455, -1.5280],
          [-2.0357, -2.0357, -2.0357,  ..., -1.5630, -1.5455, -1.5455]],

         [[-0.8458, -0.9678, -1.1247,  ...,  0.9145,  0.8797,  1.0539],
          [-0.8284, -0.9330, -

In [0]:
model = models.densenet121(pretrained=True) # densenet201 initially

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [0]:
# Freeze parameters so we don't backprop through them
from torch.optim.lr_scheduler import StepLR
for param in model.parameters():
  
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 1000)), # Change depths for densenet201
                          ('relu1', nn.ReLU()),
                          ('dropout1', nn.Dropout(p=0.2)),
                          ('fc2', nn.Linear(1000, 512)),
                          ('relu2', nn.ReLU()),
                          ('dropout2', nn.Dropout(p=0.3)),
                          ('fc3', nn.Linear(512, 102))
                          ]))

# ('output', nn.LogSoftmax(dim=1)) use this only as last structure in the classifier when not using SGD as optimizer
    
model.classifier = classifier

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.classifier.parameters(), lr=0.01, momentum=0.2) # momentum for SGD

# New variable
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
n_epochs = 200
steps = 0
every_step = 1
train_counter = 0
train_loss = 0.0
valid_loss_min = np.Inf

for epoch in range(1, n_epochs+1):
    
    steps += 1
    
    ###################
    # train the model #
    ###################
    
    for data, target in trainLoader:
      
      train_counter += 1
      print("Currect Training Batch: ", train_counter)

      data,target = data.to(device),target.to(device)

      optimizer.zero_grad()

      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      #optimizer.step()
      scheduler.step()

      train_loss += loss.item()
        
    ###################### 
    # validate the model #
    ######################

    if steps % every_step == 0:
      model.eval()
      
      valid_loss = 0.0
      accuracy = 0
      
      with torch.no_grad():
      
        for data, target in validLoader:

            validation_counter += 1
            print("Current Validating Batch: ", validation_counter)

            data,target = data.to(device),target.to(device)

            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()

            # Calculate accuracy
            ps = torch.exp(output)
            #top_p, top_class = ps.topk(1, dim=1)
            #equals = top_class == target.view(*top_class.shape)
            #accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            top_class = torch.exp(output).topk(1, dim=1)[1] 
            equals = (top_class.view(target.shape[0]) == target).type(torch.FloatTensor)
            accuracy += torch.sum(equals)

      # calculating average losses
      train_loss = train_loss/len(trainLoader)
      valid_loss = valid_loss/len(validLoader)
      final_accuracy = accuracy/len(validLoader)

      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tAccuracy: {:.6f} '.format(
          epoch, train_loss, valid_loss, final_accuracy))

      # save model if validation loss has decreased
      if valid_loss <= valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
          valid_loss_min,
          valid_loss))
          torch.save(model.state_dict(), 'drive/My Drive/AI/flower_102_model.pt')
          valid_loss_min = valid_loss
          
      train_loss = 0.0
      model.train()
      
      validation_counter = 0

Currect Training Batch:  1
Currect Training Batch:  2
Currect Training Batch:  3
Currect Training Batch:  4
Currect Training Batch:  5
Currect Training Batch:  6
Currect Training Batch:  7
Currect Training Batch:  8
Currect Training Batch:  9
Currect Training Batch:  10
Currect Training Batch:  11
Currect Training Batch:  12
Currect Training Batch:  13
Currect Training Batch:  14
Currect Training Batch:  15
Currect Training Batch:  16
Currect Training Batch:  17
Currect Training Batch:  18
Currect Training Batch:  19
Currect Training Batch:  20
Currect Training Batch:  21
Currect Training Batch:  22
Currect Training Batch:  23
Currect Training Batch:  24
Currect Training Batch:  25
Currect Training Batch:  26
Currect Training Batch:  27
Currect Training Batch:  28
Currect Training Batch:  29
Currect Training Batch:  30
Currect Training Batch:  31
Currect Training Batch:  32
Currect Training Batch:  33
Currect Training Batch:  34
Currect Training Batch:  35
Currect Training Batch:  36
C

KeyboardInterrupt: ignored

In [0]:
def savemodel(model,model_save_name,trainset):
 model.class_to_idx =trainset.class_to_idx
 #path=F"/content/gdrive/myDrive/{model_save_name}"
 torch.save({'arch': 'resnet152',
           'state_dict': model.state_dict(),
           'class_to_idx': model.class_to_idx},
           model_save_name)

In [0]:
def calculate_accuracy(model, dataloader):
    model.eval()
    with torch.no_grad():
        total_predictions_vector = None
        counter=0
        for index, (inputs, labels) in enumerate(dataloader):
            inputs.to(torch.device("cpu"))
            labels.to(torch.device("cpu"))
            counter+=1
            print(counter)
            outputs = model(inputs)
            _, y_hat = outputs.max(1)
            current_batch_predictions_vector = (y_hat==labels.data)
            if index == 0:
                total_predictions_vector = current_batch_predictions_vector
            else:
              total_predictions_vector = torch.cat(( current_batch_predictions_vector, total_predictions_vector), 0)

        total_predictions_vector = total_predictions_vector.cpu()
        valid_items = total_predictions_vector.sum().numpy()
        total_items = list(total_predictions_vector.size())[0]
        accuracy = valid_items/total_items
        return accuracy

In [0]:
filepath = "drive/My Drive/AI/flower_102_model.pt"
state_dict = torch.load(filepath, map_location='cpu')
model.load_state_dict(state_dict, strict=False)

In [0]:
calculate_accuracy(model, validLoader)

1


RuntimeError: ignored